In [1]:
! pip install imdbPY

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 6.1 MB/s eta 0:00:00:00:01


In [ ]:
import imdb
import csv
import pandas as pd
from imdb import IMDb, IMDbDataAccessError

In [ ]:
ia = imdb.IMDb()
csv_file = '/kaggle/input/imdbdata/imdb_data.csv'

In [ ]:
data=pd.read_csv(r'/kaggle/input/movielens/movies.csv')

In [ ]:
#old verison 
import imdb
import csv
import random
import pandas as pd
import concurrent.futures

# Load the movie data
imdbs = data['imdbId']

# Set up the IMDb API
ia = imdb.IMDb()

# Define the batch size for processing
batch_size = 100

# Define the function to get movie data
def get_movie_data(movie_id):
    try:
        movie = ia.get_movie(movie_id)
        ia.update(movie, 'reviews')
        ia.update(movie, 'cast')
        data = {
            'film_name': movie.get('title', ''),
            'film_rate': movie.get('rating', 0.0),  # default to 0.0 if no rating
            'plot': movie.get('plot', ''),
            'cover_url': movie.get('cover url', ''),
            'cast': ', '.join([person['name'] for person in movie.get('cast', [])[:3]]) if movie.get('cast') else '',  # return empty string if cast is not available
            'reviews': movie.get('reviews', [{}])[0].get('content', '')  # default to empty string if no reviews
        }
        return data
    except IMDbDataAccessError as e:
        return {'film_name': '', 'film_rate': 0.0, 'plot': '', 'cover_url': '', 'cast': '', 'reviews': ''}  # return default values
    except urllib.error.HTTPError as e:
        return {'film_name': '', 'film_rate': 0.0, 'plot': '', 'cover_url': '', 'cast': '', 'reviews': ''}  # return default values

# Define the function to process a batch of movie IDs
def process_batch(movie_ids):
    data_list = []
    for movie_id in movie_ids:
        data_list.append(get_movie_data(movie_id))
    return data_list

# Create a list to store the processed data
data_list = []

# Create a ThreadPoolExecutor to parallelize the API calls
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Split the movie IDs into batches
    batches = [imdbs[i:i + batch_size] for i in range(0, len(imdbs), batch_size)]
    
    # Process each batch in parallel
    futures = [executor.submit(process_batch, batch) for batch in batches]
    
    # Collect the results
    for future in concurrent.futures.as_completed(futures):
        data_list.extend(future.result())
csv_file='/kaggle/input/imdbdata/imdb_data.csv'
# Save the processed data to a CSV file
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['film_name', 'film_rate', 'plot', 'cover_url', 'cast', 'reviews'])
    for data in data_list:
        writer.writerow(list(data.values()))

# updated version with more features 

In [ ]:
import imdb
import csv
import random
import pandas as pd
import concurrent.futures
import urllib.error


# Load the movie data
imdbs = data['imdbId']

# Set up the IMDb API
ia = imdb.IMDb()

# Define the batch size for processing
batch_size = 100

# Define the function to get movie data
def get_movie_data(movie_id):
    try:
        movie = ia.get_movie(movie_id)
        ia.update(movie, 'reviews')
        ia.update(movie, 'cast')
        ia.update(movie, 'directors')
        ia.update(movie, 'writers')

        data = {
            'imdbId': movie_id,
            'film_name': movie.get('title', ''),
            'film_rate': movie.get('rating', 0.0),  # default to 0.0 if no rating
            'plot': movie.get('plot', [''])[0],  # take the first plot synopsis
            'cover_url': movie.get('cover url', ''),
            'cast': ', '.join([person['name'] for person in movie.get('cast', [])[:3]]) if movie.get('cast') else '',  # top 3 cast
            'reviews': movie.get('reviews', [{}])[0].get('content', ''),  # first review content
            'director': ', '.join([person['name'] for person in movie.get('directors', [])]) if movie.get('directors') else '',
            'writer': ', '.join([person.get('name', '') for person in movie.get('writers', [])]) if movie.get('writers') else '',
            'genre': ', '.join(movie.get('genres', [])) if movie.get('genres') else ''  # join genre list
        }
        return data
    except imdb.IMDbDataAccessError as e:
        return {'imdbId': movie_id, 'film_name': '', 'film_rate': 0.0, 'plot': '', 'cover_url': '', 'cast': '', 'reviews': '', 'director': '', 'writer': '', 'genre': ''}
    except urllib.error.HTTPError as e:
        return {'imdbId': movie_id, 'film_name': '', 'film_rate': 0.0, 'plot': '', 'cover_url': '', 'cast': '', 'reviews': '', 'director': '', 'writer': '', 'genre': ''}

# Define the function to process a batch of movie IDs
def process_batch(movie_ids):
    data_list = []
    for movie_id in movie_ids:
        data_list.append(get_movie_data(movie_id))
    return data_list

# Create a list to store the processed data
data_list = []

# Create a ThreadPoolExecutor to parallelize the API calls
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Split the movie IDs into batches
    batches = [imdbs[i:i + batch_size] for i in range(0, len(imdbs), batch_size)]
    
    # Process each batch in parallel
    futures = [executor.submit(process_batch, batch) for batch in batches]
    
    # Collect the results
    for future in concurrent.futures.as_completed(futures):
        data_list.extend(future.result())

# Specify the path to save the CSV
csv_file = '/kaggle/input/imdbdata/imdb_data.csv'

# Save the processed data to a CSV file
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['imdbId', 'film_name', 'film_rate', 'plot', 'cover_url', 'cast', 'reviews', 'director', 'writer', 'genre'])
    for data in data_list:
        writer.writerow([data['imdbId'], data['film_name'], data['film_rate'], data['plot'], data['cover_url'], data['cast'], data['reviews'], data['director'], data['writer'], data['genre']])


# missing features only 

In [ ]:
import imdb
import csv
import pandas as pd
import concurrent.futures
import urllib.error

imdbs = data['imdbId']

ia = imdb.IMDb()

batch_size = 100

# Define the function to get movie data
def get_movie_data(movie_id):
    try:
        movie = ia.get_movie(movie_id)
        ia.update(movie, info=['directors', 'genres'])  # Only updating required fields

        data = {
            'imdbId': movie_id,
            'film_name': movie.get('title', ''),
            'director': ', '.join([person['name'] for person in movie.get('directors', [])]) if movie.get('directors') else '',
            'genre': ', '.join(movie.get('genres', [])) if movie.get('genres') else ''
        }
        return data
    except imdb.IMDbDataAccessError as e:
        # Handle IMDb Data Access Error
        return {'imdbId': movie_id, 'film_name': '', 'director': '', 'genre': ''}
    except urllib.error.HTTPError as e:
        # Handle HTTP Error
        return {'imdbId': movie_id, 'film_name': '', 'director': '', 'genre': ''}

# Define the function to process a batch of movie IDs
def process_batch(movie_ids):
    data_list = []
    for movie_id in movie_ids:
        data_list.append(get_movie_data(movie_id))
    return data_list

# Create a list to store the processed data
data_list = []

# Create a ThreadPoolExecutor to parallelize the API calls
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Split the movie IDs into batches
    batches = [imdbs[i:i + batch_size] for i in range(0, len(imdbs), batch_size)]
    
    # Process each batch in parallel
    futures = [executor.submit(process_batch, batch) for batch in batches]
    
    # Collect the results
    for future in concurrent.futures.as_completed(futures):
        data_list.extend(future.result())

# Specify the path to save the CSV
csv_file = '/kaggle/input/imdbdata/imdb_data.csv'

# Save the processed data to a CSV file with only imdbId, film_name, director, and genre
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['imdbId', 'film_name', 'director', 'genre'])  # Header row
    for data in data_list:
        writer.writerow([data['imdbId'], data['film_name'], data['director'], data['genre']])


In [ ]:
# Specify the path to save the CSV in the writable /kaggle/working directory
csv_file = '/kaggle/working/imdb_data.csv'  # Change the directory to /kaggle/working

# Save the processed data to a CSV file with only imdbId, film_name, director, and genre
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['imdbId', 'film_name', 'director', 'genre'])  # Header row
    for data in data_list:
        writer.writerow([data['imdbId'], data['film_name'], data['director'], data['genre']])


# handling data more effectivly

In [ ]:
import imdb
import csv
import pandas as pd
import concurrent.futures
import urllib.error
import time
from tqdm import tqdm  # For progress tracking

# Load the movie IDs (Assuming 'data' is a DataFrame that contains imdbId)
imdbs = data['imdbId']

# Set up the IMDb API
ia = imdb.IMDb()

# Define the batch size for processing
batch_size = 100

# Define the function to get movie data with retry logic
def get_movie_data(movie_id, retries=3):
    for _ in range(retries):
        try:
            movie = ia.get_movie(movie_id)

            # Fetch the required data without explicitly updating 'directors' or 'genres'
            data = {
                'imdbId': movie_id,
                'film_name': movie.get('title', ''),
                'director': ', '.join([person['name'] for person in movie.get('directors', [])]) if movie.get('directors') else '',
                'genre': ', '.join(movie.get('genres', [])) if movie.get('genres') else ''
            }
            return data
        except (imdb.IMDbDataAccessError, urllib.error.HTTPError) as e:
            time.sleep(1)  # Wait for 1 second before retrying
    # If retries are exhausted, return empty data
    return {'imdbId': movie_id, 'film_name': '', 'director': '', 'genre': ''}

# Define the function to process a batch of movie IDs
def process_batch(movie_ids):
    data_list = []
    for movie_id in movie_ids:
        data_list.append(get_movie_data(movie_id))
    return data_list

# Create a list to store the processed data
data_list = []

# Create a ThreadPoolExecutor to parallelize the API calls
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Split the movie IDs into batches
    batches = [imdbs[i:i + batch_size] for i in range(0, len(imdbs), batch_size)]
    
    # Process each batch in parallel and use tqdm for progress tracking
    futures = [executor.submit(process_batch, batch) for batch in batches]
    
    # Collect the results with progress tracking
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="Processing batches"):
        data_list.extend(future.result())

# Specify the path to save the CSV
csv_file = '/kaggle/input/imdbdata/imdb_data.csv'

# Save the processed data to a CSV file with only imdbId, film_name, director, and genre
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['imdbId', 'film_name', 'director', 'genre'])  # Header row
    for data in data_list:
        writer.writerow([data['imdbId'], data['film_name'], data['director'], data['genre']])


# updated needed features to current file

In [ ]:
import imdb
import pandas as pd
import concurrent.futures

# Extract the list of IMDb IDs
imdbs = data['imdbId']

# Set up the IMDb API
ia = imdb.IMDb()

# Define the batch size for processing
batch_size = 100

# Define the function to get movie data
def get_movie_data(movie_id):
    try:
        movie = ia.get_movie(movie_id)
        ia.update(movie, 'reviews')
        ia.update(movie, 'cast')
        ia.update(movie, 'directors')
        ia.update(movie, 'writers')

        # Return the new data, using `.get()` to safely handle missing fields
        data = {
            'imdbId': movie_id,
            'director': ', '.join([person.get('name', '') for person in movie.get('directors', [])]) if movie.get('directors') else '',
            'writer': ', '.join([person.get('name', '') for person in movie.get('writers', [])]) if movie.get('writers') else '',
            'genre': ', '.join(movie.get('genres', [])) if movie.get('genres') else ''  # join genre list
        }
        return data
    except imdb.IMDbDataAccessError as e:
        return {'imdbId': movie_id, 'director': '', 'writer': '', 'genre': ''}
    
# Define the function to process a batch of movie IDs
def process_batch(movie_ids):
    data_list = []
    for movie_id in movie_ids:
        data_list.append(get_movie_data(movie_id))
    return data_list

# Create a list to store the processed data
data_list = []

# Create a ThreadPoolExecutor to parallelize the API calls
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Split the movie IDs into batches
    batches = [imdbs[i:i + batch_size] for i in range(0, len(imdbs), batch_size)]
    
    # Process each batch in parallel
    futures = [executor.submit(process_batch, batch) for batch in batches]
    
    # Collect the results
    for future in concurrent.futures.as_completed(futures):
        data_list.extend(future.result())

# Convert the collected data into a DataFrame
new_data_df = pd.DataFrame(data_list)

# Merge the new data with the existing dataset
updated_data = pd.merge(data, new_data_df, on='imdbId', how='left')

# Save the updated data back to the CSV file
updated_data.to_csv(csv_file, index=False)


In [ ]:
data_list

# download the csv file

In [ ]:
import pandas as pd

# Create a Pandas DataFrame from the data_list
df = pd.DataFrame(data_list)

df.to_csv('imdb_data.csv', index=False)
from IPython.display import FileLink

# This will create a download link for the file
FileLink('imdb_data.csv')

# imdb reccomendation

In [2]:
import imdb
import pandas as pd
import concurrent.futures

# Set up the IMDb API
ia = imdb.IMDb()

# Define the function to get movie data based on movie title or IMDb ID
def get_movie_id(user_input):
    # Try to interpret input as an IMDb ID (numeric)
    try:
        movie_id = int(user_input)
        movie = ia.get_movie(movie_id)
    except ValueError:
        # If not a number, assume it's a movie title and search by title
        search_results = ia.search_movie(user_input)
        if search_results:
            # Take the first search result as the most likely match
            movie = search_results[0]
        else:
            return None  # Return None if no results found
    
    return movie.movieID  # Return the movie ID

# Define the function to get movie recommendations
def get_recommendations(movie_id):
    try:
        movie = ia.get_movie(movie_id)
        ia.update(movie, 'recommendations')  # Fetch recommendations

        # Get the recommended movies (up to 3 recommendations)
        recommended_movies = movie.get('recommendations', [])[:3]
        recommended_titles = ', '.join([rec.get('title', '') for rec in recommended_movies]) if recommended_movies else 'No recommendations'

        # Return the top 3 recommendations
        return {
            'imdbId': movie_id,
            'recommended_movies': recommended_titles  # Add the top 3 recommendations
        }
    except imdb.IMDbDataAccessError:
        return {'imdbId': movie_id, 'recommended_movies': 'No recommendations'}

# User input function to accept either movie title or IMDb ID
def get_user_recommendations():
    user_input = input("Enter a movie name or IMDb ID: ")

    # Normalize user input (case insensitive, remove extra spaces)
    user_input_normalized = user_input.strip().lower()

    # Get the IMDb movie ID
    movie_id = get_movie_id(user_input_normalized)
    if movie_id:
        # Fetch recommendations if the movie ID is valid
        recommendations = get_recommendations(movie_id)
        print(f"Top 3 recommended movies for {user_input}:")
        print(recommendations['recommended_movies'])
    else:
        print("Sorry, no movie found with that name or IMDb ID.")

# Run the recommendation system for user input
get_user_recommendations()


Enter a movie name or IMDb ID:  heat


Top 3 recommended movies for heat:
No recommendations


In [7]:
get_user_recommendations()

Enter a movie name or IMDb ID:  heat


Top 3 recommended movies for heat:


TypeError: string indices must be integers

In [13]:
import imdb

# Set up the IMDb API
ia = imdb.IMDb()

# Function to get movie recommendations
def get_recommendations(movie_input):
    try:
        # Check if input is numeric (IMDb ID) or a movie name
        if movie_input.isdigit():  # IMDb ID
            movie = ia.get_movie(movie_input)
        else:  # Movie name
            search_results = ia.search_movie(movie_input)
            if not search_results:
                return f"No movie found for '{movie_input}'"
            movie = search_results[0]  # Take the first search result

        # Get movie title
        movie_title = movie.get('title', movie_input)

        # Try to update with recommendations
        ia.update(movie, 'recommendations')
        recommended_movies = movie.get('recommendations', [])

        # If recommendations are missing, provide more feedback
        if not recommended_movies:
            return f"No recommendations available for '{movie_title}'. The IMDb API may not have this data."

        # Format the top 3 recommendations
        recommended_titles = ', '.join([rec.get('title', '') for rec in recommended_movies[:3]])
        return f"Top 3 recommended movies for {movie_title}:\n{recommended_titles}"

    except imdb.IMDbDataAccessError:
        return f"Error accessing IMDb data for '{movie_input}'"
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Main interaction loop
def main():
    movie_input = input("Enter a movie name or IMDb ID: ").strip()

    # Get and display recommendations
    result = get_recommendations(movie_input)
    print(result)

# Run the main function
if __name__ == "__main__":
    main()


Enter a movie name or IMDb ID:  inception


No recommendations available for 'Inception'. The IMDb API may not have this data.


In [33]:
import imdb

# Set up the IMDb API
ia = imdb.IMDb()

# Function to get movie recommendations or similar movies by keywords/genres
def get_similar_movies(movie_input):
    try:
        # Check if input is numeric (IMDb ID) or a movie name
        if movie_input.isdigit():  # IMDb ID
            movie = ia.get_movie(movie_input)
        else:  # Movie name
            search_results = ia.search_movie(movie_input)
            if not search_results:
                return f"No movie found for '{movie_input}'"
            movie = search_results[0]  # Take the first search result

        # Get movie title
        movie_title = movie.get('title', movie_input)
        print(f"Movie found: {movie_title}")

        # First, try to get recommendations
        ia.update(movie, 'recommendations')
        recommended_movies = movie.get('recommendations', [])
        print(f"Recommendations: {recommended_movies}")

        # If recommendations are available, use them
        if recommended_movies:
            recommended_titles = ', '.join([rec.get('title', '') for rec in recommended_movies[:3]])
            return f"Top 3 recommended movies for '{movie_title}':\n{recommended_titles}"

        # Access genres and keywords directly from the movie object
        genres = movie.get('genres', [])
        keywords = movie.get('keywords', [])
        print(f"Genres: {genres}")
        print(f"Keywords: {keywords}")

        # Search for similar movies based on genre or keywords
        if genres:
            similar_titles = ', '.join([f"Similar movie {i+1}" for i in range(3)])
            return f"No recommendations available for '{movie_title}'. Similar movies by genre:\n{similar_titles}"

        if keywords:
            similar_titles = ', '.join([f"Similar movie {i+1}" for i in range(3)])
            return f"No recommendations available for '{movie_title}'. Similar movies by keyword:\n{similar_titles}"

        # If no similar movies found, return a fallback message
        return f"No recommendations or similar movies available for '{movie_title}'."

    except imdb.IMDbDataAccessError:
        return f"Error accessing IMDb data for '{movie_input}'"
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Main interaction loop
def main():
    movie_input = input("Enter a movie name or IMDb ID: ").strip()

    # Get and display similar movies or recommendations
    result = get_similar_movies(movie_input)
    print(result)

# Run the main function
if __name__ == "__main__":
    main()


Enter a movie name or IMDb ID:  me before you


Movie found: Me Before You
Recommendations: []
Genres: []
Keywords: []
No recommendations or similar movies available for 'Me Before You'.


In [35]:
import imdb

# Set up the IMDb API
ia = imdb.IMDb()

# Function to get movie recommendations or similar movies by keywords/genres
def get_similar_movies(movie_input):
    try:
        # Check if input is numeric (IMDb ID) or a movie name
        if movie_input.isdigit():  # IMDb ID
            movie = ia.get_movie(movie_input)
        else:  # Movie name
            search_results = ia.search_movie(movie_input)
            if not search_results:
                return f"No movie found for '{movie_input}'"
            movie = search_results[0]  # Take the first search result

        # Get movie title
        movie_title = movie.get('title', movie_input)
        print(f"Movie found: {movie_title}")

        # First, try to get recommendations
        ia.update(movie, 'recommendations')
        recommended_movies = movie.get('recommendations', [])
        print(f"Recommendations: {recommended_movies}")

        # If recommendations are available, use them
        if recommended_movies:
            recommended_titles = ', '.join([rec.get('title', '') for rec in recommended_movies[:3]])
            return f"Top 3 recommended movies for '{movie_title}':\n{recommended_titles}"

        # Update movie with more information
        ia.update(movie)
        genres = movie.get('genres', [])
        keywords = movie.get('keywords', [])
        print(f"Genres: {genres}")
        print(f"Keywords: {keywords}")

        # Search for similar movies based on genre or keywords
        if genres:
            similar_titles = ', '.join([f"Similar movie {i+1}" for i in range(3)])
            return f"No recommendations available for '{movie_title}'. Similar movies by genre:\n{similar_titles}"

        if keywords:
            similar_titles = ', '.join([f"Similar movie {i+1}" for i in range(3)])
            return f"No recommendations available for '{movie_title}'. Similar movies by keyword:\n{similar_titles}"

        # If no similar movies found, return a fallback message
        return f"No recommendations or similar movies available for '{movie_title}'."

    except imdb.IMDbDataAccessError:
        return f"Error accessing IMDb data for '{movie_input}'"
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Main interaction loop
def main():
    movie_input = input("Enter a movie name or IMDb ID: ").strip()

    # Get and display similar movies or recommendations
    result = get_similar_movies(movie_input)
    print(result)

# Run the main function
if __name__ == "__main__":
    main()


Enter a movie name or IMDb ID:  interstellar


Movie found: Interstellar
Recommendations: []
Genres: ['Adventure', 'Drama', 'Sci-Fi']
Keywords: []
No recommendations available for 'Interstellar'. Similar movies by genre:
Similar movie 1, Similar movie 2, Similar movie 3


In [36]:
import imdb

# Set up the IMDb API
ia = imdb.IMDb()

# Function to get movie recommendations or similar movies by keywords/genres
def get_similar_movies(movie_input):
    try:
        # Check if input is numeric (IMDb ID) or a movie name
        if movie_input.isdigit():  # IMDb ID
            movie = ia.get_movie(movie_input)
        else:  # Movie name
            search_results = ia.search_movie(movie_input)
            if not search_results:
                return f"No movie found for '{movie_input}'"
            movie = search_results[0]  # Take the first search result

        # Get movie title
        movie_title = movie.get('title', movie_input)
        print(f"Movie found: {movie_title}")

        # First, try to get recommendations
        ia.update(movie, 'recommendations')
        recommended_movies = movie.get('recommendations', [])
        print(f"Recommendations: {recommended_movies}")

        # If recommendations are available, use them
        if recommended_movies:
            recommended_titles = ', '.join([rec.get('title', '') for rec in recommended_movies[:3]])
            return f"Top 3 recommended movies for '{movie_title}':\n{recommended_titles}"

        # Update movie with more information
        ia.update(movie)
        genres = movie.get('genres', [])
        keywords = movie.get('keywords', [])
        print(f"Genres: {genres}")
        print(f"Keywords: {keywords}")

        # Search for similar movies based on genre
        if genres:
            similar_movies = []
            for genre in genres:
                genre_movies = ia.get_top50_movies_by_genres(genre)
                similar_movies.extend(genre_movies)
            similar_movies = list({movie.movieID: movie for movie in similar_movies}.values())  # Remove duplicates
            similar_titles = ', '.join([sim_movie.get('title', '') for sim_movie in similar_movies[:3]])
            return f"No recommendations available for '{movie_title}'. Similar movies by genre:\n{similar_titles}"

        # If no similar movies found, return a fallback message
        return f"No recommendations or similar movies available for '{movie_title}'."

    except imdb.IMDbDataAccessError:
        return f"Error accessing IMDb data for '{movie_input}'"
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Main interaction loop
def main():
    movie_input = input("Enter a movie name or IMDb ID: ").strip()

    # Get and display similar movies or recommendations
    result = get_similar_movies(movie_input)
    print(result)

# Run the main function
if __name__ == "__main__":
    main()


Enter a movie name or IMDb ID:  heat


Movie found: Heat
Recommendations: []
Genres: ['Action', 'Crime', 'Drama']
Keywords: []


2024-09-17 13:21:48,107 ERROR [imdbpy.parser.http.domparser] /opt/conda/lib/python3.10/site-packages/imdb/parser/http/utils.py:438: DOMHTMLSearchMovieAdvancedParser: caught exception postprocessing data
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/imdb/parser/http/utils.py", line 436, in parse
    data = self.postprocess_data(data)
  File "/opt/conda/lib/python3.10/site-packages/imdb/parser/http/searchMovieAdvancedParser.py", line 207, in postprocess_data
    data['data'] = []
TypeError: 'mappingproxy' object does not support item assignment


An error occurred: 'data'


In [37]:
import imdb

# Set up the IMDb API
ia = imdb.IMDb()

# Function to get movie recommendations or similar movies by keywords/genres
def get_similar_movies(movie_input):
    try:
        # Check if input is numeric (IMDb ID) or a movie name
        if movie_input.isdigit():  # IMDb ID
            movie = ia.get_movie(movie_input)
        else:  # Movie name
            search_results = ia.search_movie(movie_input)
            if not search_results:
                return f"No movie found for '{movie_input}'"
            movie = search_results[0]  # Take the first search result

        # Get movie title
        movie_title = movie.get('title', movie_input)
        print(f"Movie found: {movie_title}")

        # First, try to get recommendations
        ia.update(movie, 'recommendations')
        recommended_movies = movie.get('recommendations', [])
        print(f"Recommendations: {recommended_movies}")

        # If recommendations are available, use them
        if recommended_movies:
            recommended_titles = ', '.join([rec.get('title', '') for rec in recommended_movies[:3]])
            return f"Top 3 recommended movies for '{movie_title}':\n{recommended_titles}"

        # Update movie with more information
        ia.update(movie)
        genres = movie.get('genres', [])
        print(f"Genres: {genres}")

        # Search for similar movies based on genre
        if genres:
            similar_movies = []
            for genre in genres:
                search_results = ia.search_movie_advanced(genre=genre)
                similar_movies.extend(search_results[:3])  # Get top 3 movies for each genre
            similar_movies = list({movie.movieID: movie for movie in similar_movies}.values())  # Remove duplicates
            similar_titles = ', '.join([sim_movie.get('title', '') for sim_movie in similar_movies[:3]])
            return f"No recommendations available for '{movie_title}'. Similar movies by genre:\n{similar_titles}"

        # If no similar movies found, return a fallback message
        return f"No recommendations or similar movies available for '{movie_title}'."

    except imdb.IMDbDataAccessError:
        return f"Error accessing IMDb data for '{movie_input}'"
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Main interaction loop
def main():
    movie_input = input("Enter a movie name or IMDb ID: ").strip()

    # Get and display similar movies or recommendations
    result = get_similar_movies(movie_input)
    print(result)

# Run the main function
if __name__ == "__main__":
    main()


Enter a movie name or IMDb ID:  heat


Movie found: Heat
Recommendations: []
Genres: ['Action', 'Crime', 'Drama']
An error occurred: IMDbBase.search_movie_advanced() got an unexpected keyword argument 'genre'


In [38]:
import imdb

# Set up the IMDb API
ia = imdb.IMDb()

# Function to get movie recommendations or similar movies by keywords/genres
def get_similar_movies(movie_input):
    try:
        # Check if input is numeric (IMDb ID) or a movie name
        if movie_input.isdigit():  # IMDb ID
            movie = ia.get_movie(movie_input)
        else:  # Movie name
            search_results = ia.search_movie(movie_input)
            if not search_results:
                return f"No movie found for '{movie_input}'"
            movie = search_results[0]  # Take the first search result

        # Get movie title
        movie_title = movie.get('title', movie_input)
        print(f"Movie found: {movie_title}")

        # First, try to get recommendations
        ia.update(movie, 'recommendations')
        recommended_movies = movie.get('recommendations', [])
        print(f"Recommendations: {recommended_movies}")

        # If recommendations are available, use them
        if recommended_movies:
            recommended_titles = ', '.join([rec.get('title', '') for rec in recommended_movies[:3]])
            return f"Top 3 recommended movies for '{movie_title}':\n{recommended_titles}"

        # Update movie with more information
        ia.update(movie)
        genres = movie.get('genres', [])
        print(f"Genres: {genres}")

        # Search for similar movies based on genre
        if genres:
            similar_movies = []
            for genre in genres:
                search_results = ia.search_movie(genre)
                similar_movies.extend(search_results[:3])  # Get top 3 movies for each genre
            similar_movies = list({movie.movieID: movie for movie in similar_movies}.values())  # Remove duplicates
            similar_titles = ', '.join([sim_movie.get('title', '') for sim_movie in similar_movies[:3]])
            return f"No recommendations available for '{movie_title}'. Similar movies by genre:\n{similar_titles}"

        # If no similar movies found, return a fallback message
        return f"No recommendations or similar movies available for '{movie_title}'."

    except imdb.IMDbDataAccessError:
        return f"Error accessing IMDb data for '{movie_input}'"
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Main interaction loop
def main():
    movie_input = input("Enter a movie name or IMDb ID: ").strip()

    # Get and display similar movies or recommendations
    result = get_similar_movies(movie_input)
    print(result)

# Run the main function
if __name__ == "__main__":
    main()


Enter a movie name or IMDb ID:  heat


Movie found: Heat
Recommendations: []
Genres: ['Action', 'Crime', 'Drama']
No recommendations available for 'Heat'. Similar movies by genre:
Action, Last Action Hero, Back in Action


In [40]:
import imdb

# Set up the IMDb API
ia = imdb.IMDb()

# Function to get movie recommendations or similar movies by keywords/genres
def get_similar_movies(movie_input):
    try:
        # Check if input is numeric (IMDb ID) or a movie name
        if movie_input.isdigit():  # IMDb ID
            movie = ia.get_movie(movie_input)
        else:  # Movie name
            search_results = ia.search_movie(movie_input)
            if not search_results:
                return f"No movie found for '{movie_input}'"
            movie = search_results[0]  # Take the first search result

        # Get movie title
        movie_title = movie.get('title', movie_input)
        print(f"Movie found: {movie_title}")

        # First, try to get recommendations
        ia.update(movie, 'recommendations')
        recommended_movies = movie.get('recommendations', [])
        print(f"Recommendations: {recommended_movies}")

        # If recommendations are available, use them
        if recommended_movies:
            recommended_titles = ', '.join([rec.get('title', '') for rec in recommended_movies[:3]])
            return f"Top 3 recommended movies for '{movie_title}':\n{recommended_titles}"

        # Update movie with more information
        ia.update(movie)
        genres = movie.get('genres', [])
        print(f"Genres: {genres}")

        # Search for similar movies based on all genres
        similar_movies = []
        for genre in genres:
            search_results = ia.search_movie(f"{genre} movie")
            similar_movies.extend(search_results[:3])  # Get top 3 movies for each genre
        similar_movies = list({movie.movieID: movie for movie in similar_movies}.values())  # Remove duplicates
        similar_titles = ', '.join([sim_movie.get('title', '') for sim_movie in similar_movies[:3]])

        return f"No recommendations available for '{movie_title}'. Similar movies by genre:\n{similar_titles}"

    except imdb.IMDbDataAccessError:
        return f"Error accessing IMDb data for '{movie_input}'"
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Main interaction loop
def main():
    movie_input = input("Enter a movie name or IMDb ID: ").strip()

    # Get and display similar movies or recommendations
    result = get_similar_movies(movie_input)
    print(result)

# Run the main function
if __name__ == "__main__":
    main()


Enter a movie name or IMDb ID:  heat


Movie found: Heat
Recommendations: []
Genres: ['Action', 'Crime', 'Drama']
No recommendations available for 'Heat'. Similar movies by genre:
Gundam I: The Live-Action Movie, Gintama Live Action the Movie, Ryan: First Mission (Fardeen Khan, Kartik Aaryan)


In [22]:
import imdb

# Set up the IMDb API
ia = imdb.IMDb()

# Function to fetch basic movie details
def get_movie_info(movie_input):
    try:
        # Check if input is IMDb ID (numeric) or movie title (string)
        if movie_input.isdigit():
            movie = ia.get_movie(movie_input)
        else:
            search_results = ia.search_movie(movie_input)
            if not search_results:
                return f"No movie found for '{movie_input}'"
            movie = search_results[0]  # Take the first search result
        
        # Print basic information for debugging
        movie_title = movie.get('title', 'Unknown Title')
        movie_year = movie.get('year', 'Unknown Year')
        genres = movie.get('genres', [])
        print(f"Movie found: {movie_title} ({movie_year})")
        print(f"Genres: {genres}")

        return f"Movie: {movie_title} ({movie_year}) | Genres: {', '.join(genres)}"

    except imdb.IMDbDataAccessError:
        return f"Error accessing IMDb data for '{movie_input}'"
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Main interaction loop
def main():
    movie_input = input("Enter a movie name or IMDb ID: ").strip()

    # Get and display movie info
    result = get_movie_info(movie_input)
    print(result)

# Run the main function
if __name__ == "__main__":
    main()


Enter a movie name or IMDb ID:  interstellar


Movie found: Interstellar (2014)
Genres: []
Movie: Interstellar (2014) | Genres: 


# assosication

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Load the dataset in chunks
chunksize = 1000
chunks = pd.read_csv(r'/kaggle/input/rating/ratings.csv', chunksize=chunksize)

# Concatenate all chunks into a single DataFrame
ratings = pd.concat(chunks, ignore_index=True)

# Filter only ratings of 4.0 and above for "liked" movies
ratings_filtered = ratings[ratings['rating'] >= 4.0]

# Create a user-item matrix
user_movie_matrix = ratings_filtered.pivot_table(index='userId', columns='movieId', aggfunc='size', fill_value=0)

# Apply FP-Growth to find frequent item sets
frequent_itemsets = fpgrowth(user_movie_matrix, min_support=0.02, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Sort the rules by lift
rules = rules.sort_values(by='lift', ascending=False)

# Display the rules
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


# assosiation model 

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Load the dataset in chunks
chunksize = 1000
chunks = pd.read_csv(r'/kaggle/input/rating/ratings.csv', chunksize=chunksize)

# Concatenate all chunks into a single DataFrame
ratings = pd.concat(chunks, ignore_index=True)

# Filter only ratings of 4.0 and above for "liked" movies
ratings_filtered = ratings[ratings['rating'] >= 4.0]

# Create a user-item matrix, where 1 represents a "liked" movie (rating 4.0 or above)
user_movie_matrix = ratings_filtered.pivot_table(index='userId', columns='movieId', aggfunc='size', fill_value=0)

# Apply FP-Growth to find frequent item sets
frequent_itemsets = fpgrowth(user_movie_matrix, min_support=0.02, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Sort the rules by lift
rules = rules.sort_values(by='lift', ascending=False)

# Load the movies dataset to map movie IDs to movie titles
movies = pd.read_csv(r'/kaggle/input/movielens/movies.csv')

# Create a function to get movie recommendations
def get_movie_id(movie_title):
    """
    Get the movieId for the given movie title from the movies dataset.
    """
    movie_id = movies[movies['title'].str.contains(movie_title, case=False, na=False)]['movieId']
    if len(movie_id) > 0:
        return movie_id.values[0]
    else:
        return None

def recommend_movies(movie_title, rules, movies, top_n=3):
    """
    Recommend associated movies based on the input movie title.
    """
    movie_id = get_movie_id(movie_title)
    if movie_id is None:
        return f"Movie '{movie_title}' not found in the dataset."

    # Find rules where the input movie is in the antecedents
    matching_rules = rules[rules['antecedents'].apply(lambda x: movie_id in x)]

    if matching_rules.empty:
        return f"No associated movies found for '{movie_title}'."

    # Extract the consequents from the association rules
    movie_recommendations = []
    for _, row in matching_rules.iterrows():
        for consequent in row['consequents']:
            if consequent != movie_id:
                movie_recommendations.append((consequent, row['lift']))

    # Remove duplicates and sort by lift value (descending)
    movie_recommendations = sorted(list(set(movie_recommendations)), key=lambda x: x[1], reverse=True)

    # Get the top N movie recommendations
    recommended_movie_ids = [rec[0] for rec in movie_recommendations[:top_n]]
    recommended_movies = movies[movies['movieId'].isin(recommended_movie_ids)]['title'].values

    return recommended_movies

# Example usage:
movie_name = 'Interstellar'  # Change this to the movie title you want recommendations for
recommended_movies = recommend_movies(movie_name, rules, movies)

if isinstance(recommended_movies, str):
    print(recommended_movies)  # If the function returns an error message
else:
    print(f"Movies recommended based on '{movie_name}':")
    for i, movie in enumerate(recommended_movies, 1):
        print(f"{i}. {movie}")


# assosiation model low resources

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules
from scipy.sparse import csr_matrix

# Load the dataset in chunks to avoid memory overload
chunksize = 10**6  # Increase the chunksize to process more data at once but keep it manageable
chunks = pd.read_csv(r'/kaggle/input/rating/ratings.csv', chunksize=chunksize, usecols=['userId', 'movieId', 'rating'])

# Initialize an empty list to store filtered data
filtered_chunks = []

# Process each chunk individually
for chunk in chunks:
    # Filter ratings of 4.0 and above for "liked" movies
    filtered_chunk = chunk[chunk['rating'] >= 4.0]
    filtered_chunks.append(filtered_chunk)

# Concatenate all filtered chunks into a single DataFrame
ratings_filtered = pd.concat(filtered_chunks, ignore_index=True)

# Create a user-item matrix in sparse format to save memory
user_movie_matrix = ratings_filtered.pivot_table(index='userId', columns='movieId', aggfunc='size', fill_value=0)
user_movie_matrix_sparse = csr_matrix(user_movie_matrix.values)

# Apply FP-Growth to find frequent item sets
frequent_itemsets = fpgrowth(pd.DataFrame(user_movie_matrix_sparse.todense(), columns=user_movie_matrix.columns), min_support=0.02, use_colnames=True)

# Generate association rules from the frequent item sets
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Sort the rules by lift to find the most interesting ones
rules = rules.sort_values(by='lift', ascending=False)

# Load the movies dataset to map movie IDs to movie titles
movies = pd.read_csv(r'/kaggle/input/movielens/movies.csv')

# Create a function to get movie recommendations
def get_movie_id(movie_title):
    """
    Get the movieId for the given movie title from the movies dataset.
    """
    movie_id = movies[movies['title'].str.contains(movie_title, case=False, na=False)]['movieId']
    if len(movie_id) > 0:
        return movie_id.values[0]
    else:
        return None

def recommend_movies(movie_title, rules, movies, top_n=3):
    """
    Recommend associated movies based on the input movie title.
    """
    movie_id = get_movie_id(movie_title)
    if movie_id is None:
        return f"Movie '{movie_title}' not found in the dataset."

    # Find rules where the input movie is in the antecedents
    matching_rules = rules[rules['antecedents'].apply(lambda x: movie_id in x)]

    if matching_rules.empty:
        return f"No associated movies found for '{movie_title}'."

    # Extract the consequents from the association rules
    movie_recommendations = []
    for _, row in matching_rules.iterrows():
        for consequent in row['consequents']:
            if consequent != movie_id:
                movie_recommendations.append((consequent, row['lift']))

    # Remove duplicates and sort by lift value (descending)
    movie_recommendations = sorted(list(set(movie_recommendations)), key=lambda x: x[1], reverse=True)

    # Get the top N movie recommendations
    recommended_movie_ids = [rec[0] for rec in movie_recommendations[:top_n]]
    recommended_movies = movies[movies['movieId'].isin(recommended_movie_ids)]['title'].values

    return recommended_movies

# Example usage:
movie_name = 'Interstellar'  # Change this to the movie title you want recommendations for
recommended_movies = recommend_movies(movie_name, rules, movies)

if isinstance(recommended_movies, str):
    print(recommended_movies)  # If the function returns an error message
else:
    print(f"Movies recommended based on '{movie_name}':")
    for i, movie in enumerate(recommended_movies, 1):
        print(f"{i}. {movie}")


# final try 

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules
from scipy.sparse import csr_matrix
import re

# Function to clean movie titles by removing special characters and lowercasing
def clean_movie_title(title):
    # Remove anything in parentheses (like year)
    title = re.sub(r'\(.*?\)', '', title)
    # Remove extra spaces and lowercase the title
    title = re.sub(r'\s+', ' ', title).strip().lower()
    return title

# Load the dataset in chunks
chunksize = 10**6
chunks = pd.read_csv(r'/kaggle/input/rating/ratings.csv', chunksize=chunksize, usecols=['userId', 'movieId', 'rating'])

# Initialize an empty list to store filtered data
filtered_chunks = []

# Process each chunk individually
for chunk in chunks:
    # Filter ratings of 4.0 and above for "liked" movies
    filtered_chunk = chunk[chunk['rating'] >= 4.0]
    filtered_chunks.append(filtered_chunk)

# Concatenate filtered chunks
ratings_filtered = pd.concat(filtered_chunks, ignore_index=True)

# Create a user-item matrix in sparse format
user_movie_matrix = ratings_filtered.pivot_table(index='userId', columns='movieId', aggfunc='size', fill_value=0)
user_movie_matrix_sparse = csr_matrix(user_movie_matrix.values)

# Apply FP-Growth to find frequent item sets
frequent_itemsets = fpgrowth(pd.DataFrame(user_movie_matrix_sparse.todense(), columns=user_movie_matrix.columns), min_support=0.05, use_colnames=True)

# Generate association rules from frequent item sets
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Sort rules by lift
rules = rules.sort_values(by='lift', ascending=False)

# Load the movies dataset
movies = pd.read_csv(r'/kaggle/input/movielens/movies.csv')

# Apply the clean_movie_title function to standardize movie titles
movies['clean_title'] = movies['title'].apply(clean_movie_title)

# Create a function to get movie recommendations
def get_movie_id(movie_title):
    """
    Get the movieId for the given movie title from the movies dataset.
    Clean the input movie title for consistency.
    """
    cleaned_title = clean_movie_title(movie_title)
    movie_id = movies[movies['clean_title'] == cleaned_title]['movieId']
    
    if len(movie_id) > 0:
        return movie_id.values[0]
    else:
        return None

def recommend_movies(movie_title, rules, movies, top_n=3):
    """
    Recommend associated movies based on the input movie title.
    """
    movie_id = get_movie_id(movie_title)
    if movie_id is None:
        return f"Movie '{movie_title}' not found in the dataset."

    # Find rules where the input movie is in the antecedents
    matching_rules = rules[rules['antecedents'].apply(lambda x: movie_id in x)]

    if matching_rules.empty:
        return f"No associated movies found for '{movie_title}'."

    # Extract the consequents from the association rules
    movie_recommendations = []
    for _, row in matching_rules.iterrows():
        for consequent in row['consequents']:
            if consequent != movie_id:
                movie_recommendations.append((consequent, row['lift']))

    # Remove duplicates and sort by lift value
    movie_recommendations = sorted(list(set(movie_recommendations)), key=lambda x: x[1], reverse=True)

    # Get top N movie recommendations
    recommended_movie_ids = [rec[0] for rec in movie_recommendations[:top_n]]
    recommended_movies = movies[movies['movieId'].isin(recommended_movie_ids)]['title'].values

    return recommended_movies



In [ ]:
# Example usage:
movie_name = 'Heat (1995)'  # You can try different variations like 'interstellar'
recommended_movies = recommend_movies(movie_name, rules, movies)

if isinstance(recommended_movies, str):
    print(recommended_movies)  # Error message
else:
    print(f"Movies recommended based on '{movie_name}':")
    for i, movie in enumerate(recommended_movies, 1):
        print(f"{i}. {movie}")

# cosine matrix improved

In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the movie dataset
data = pd.read_csv(r'D:\abdo\AI\projects\recommender\needed features\imdb_data.csv')

# Convert data to a DataFrame
df = pd.DataFrame(data)

# Replace NaN values in relevant columns with an empty string
df['overview'] = df['overview'].fillna('')
df['reviews'] = df['reviews'].fillna('')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('')
df['genre'] = df['genre'].fillna('')

# Combine 'title', 'overview', 'cast', 'director', 'genre', and 'reviews' for similarity comparison
df['combined_features'] = df['title'] + ' ' + df['overview'] + ' ' + df['cast'] + ' ' + df['director'] + ' ' + df['genre'] + ' ' + df['reviews']

# Text Vectorization using TF-IDF (including bi-grams for better context)
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))
tfidf_matrix = tfidf.fit_transform(df['combined_features'].fillna(''))

# Calculate cosine similarity between all movies
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Helper function to extract movie title
def extract_title(title):
    if pd.isna(title):
        return ''
    title = title.lower()
    title = re.sub(r'\s*\(\d{4}\)$', '', title)  # Remove year information
    return title

# Define the function to recommend movies
def recommend_movies(movie_title, cosine_sim=cosine_sim, df=df):
    movie_title_clean = extract_title(movie_title)
    
    # Check if the movie title exists in the dataset
    if movie_title_clean in df['title'].apply(extract_title).values:
        # Get the index of the movie that matches the title
        idx = df.index[df['title'].apply(extract_title) == movie_title_clean][0]
        
        # Get the pairwise similarity scores of all movies with that movie
        sim_scores = list(enumerate(cosine_sim[idx]))
        
        # Sort the movies based on similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Get the top 3 most similar movies (excluding the input movie itself)
        recommended_movies = []
        for score in sim_scores[1:]:
            movie_data = df.iloc[score[0]]
            recommended_movies.append({
                'title': movie_data['title'],
                'film_rate': movie_data['film_rate'],
                'imdbId': movie_data['imdbId'],
                'cover_url': movie_data['cover_url'],
                'director': movie_data['director'],
                'genre': movie_data['genre'],
                'cast': movie_data['cast'],
                'overview': movie_data['overview'],
                'reviews': movie_data['reviews']
            })
            if len(recommended_movies) == 3:  # Limit to top 3 movies
                break
        
        # Return the top 3 most similar movies with all requested details
        return recommended_movies
    else:
        return []  # Return an empty list if the movie is not found

# Example usage: Recommend movies based on the title "Heat"
test = 'heat'
recommended_movies = recommend_movies(test)
print(f"Movies recommended based on '{test}':")
for movie in recommended_movies:
    print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, IMDb ID: {movie['imdbId']}")
    print(f"Director: {movie['director']}, Genre: {movie['genre']}")
    print(f"Cast: {movie['cast']}")
    print(f"Overview: {movie['overview']}")
    print(f"Reviews: {movie['reviews']}")
    print(f"Cover URL: {movie['cover_url']}")
    print("-" * 80)


# LSTM

In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


# lstm 256 patch size adam w 

In [ ]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics.pairwise import cosine_similarity

# Load the movie dataset
data = pd.read_csv(r'/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Preprocessing
df = pd.DataFrame(data)
df['overview'] = df['overview'].fillna('')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('(no data about the director is found)')
df['genre'] = df['genre'].fillna('')
df['tag'] = df['tag'].fillna('')
df['reviews'] = df['reviews'].fillna('')

# Combine features into a single column for text processing
df['combined_features'] = df['overview'] + ' ' + df['cast'] + ' ' + df['director'] + ' ' + df['genre'] + ' ' + df['tag'] + ' ' + df['reviews']

# Tokenization and padding
max_words = 10000
max_sequence_len = 300

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['combined_features'])
sequences = tokenizer.texts_to_sequences(df['combined_features'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='post', truncating='post')

# Define LSTM model
embedding_dim = 100

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_sequence_len))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='AdamW', metrics=['accuracy'])
model.summary()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['rating'], test_size=0.2, random_state=42)

# Train the LSTM model
model.fit(X_train, y_train, epochs=5, batch_size=256, validation_data=(X_test, y_test))

# Generate embeddings from the trained LSTM model
embedding_layer = Sequential()
embedding_layer.add(model.layers[0])
embedding_layer.add(model.layers[1])
embedding_layer.compile(loss='binary_crossentropy', optimizer='AdamW')

# Generate embeddings for all movies
movie_embeddings = embedding_layer.predict(padded_sequences)

# Flatten the LSTM output by averaging over timesteps
movie_embeddings_2d = np.mean(movie_embeddings, axis=1)

# Calculate cosine similarity based on 2D embeddings
cosine_sim_lstm = cosine_similarity(movie_embeddings_2d, movie_embeddings_2d)

# Function to clean movie titles (remove special characters and lowercase)
def extract_title(title):
    # Check if the title is NaN or not a string, return an empty string in that case
    if isinstance(title, str):
        title_cleaned = title.lower()
        title_cleaned = re.sub(r'\W+', ' ', title_cleaned)
        return title_cleaned.strip()
    else:
        return np.nan

# Function to recommend movies based on LSTM embeddings
def recommend_movies_lstm(movie_title, cosine_sim=cosine_sim_lstm, df=df):
    movie_title_clean = extract_title(movie_title)
    
    # Check if the movie title exists in the dataset
    if movie_title_clean in df['title'].apply(extract_title).values:
        # Get the index of the movie that matches the title
        idx = df.index[df['title'].apply(extract_title) == movie_title_clean][0]
        
        # Get the pairwise similarity scores of all movies with that movie
        sim_scores = list(enumerate(cosine_sim[idx]))
        
        # Sort the movies based on similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Get the top 3 most similar movies (excluding the input movie itself)
        recommended_movies = []
        for score in sim_scores[1:]:
            movie_data = df.iloc[score[0]]
            recommended_movies.append({
                'title': movie_data['title'],
                'film_rate': movie_data['film_rate'],
                'imdbId': movie_data['imdbId'],
                'cover_url': movie_data['cover_url'],
                'director': movie_data['director'],
                'genre': movie_data['genre'],
                'cast': movie_data['cast'],
                'overview': movie_data['overview'],
                'reviews': movie_data['reviews'],
                'tag': movie_data['tag'],
                'rating': movie_data['rating']
            })
            if len(recommended_movies) == 3:  # Limit to top 3 movies
                break
        
        # Return the top 3 most similar movies with all requested details
        return recommended_movies
    else:
        return []  # Return an empty list if the movie is not found
# Example usage with LSTM model
test_movie = 'heat'
recommended_movies_lstm = recommend_movies_lstm(test_movie)

if not recommended_movies_lstm:
    print(f"'{test_movie}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie}':")
    for movie in recommended_movies_lstm:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)


# lstm 256 batch adam

In [ ]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics.pairwise import cosine_similarity

# Load the movie dataset
data = pd.read_csv(r'/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Preprocessing
df = pd.DataFrame(data)
df['overview'] = df['overview'].fillna('')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('(no data about the director is found)')
df['genre'] = df['genre'].fillna('')
df['tag'] = df['tag'].fillna('')
df['reviews'] = df['reviews'].fillna('')

# Combine features into a single column for text processing
df['combined_features'] = df['overview'] + ' ' + df['cast'] + ' ' + df['director'] + ' ' + df['genre'] + ' ' + df['tag'] + ' ' + df['reviews']

# Tokenization and padding
max_words = 10000
max_sequence_len = 300

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['combined_features'])
sequences = tokenizer.texts_to_sequences(df['combined_features'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='post', truncating='post')

# Define LSTM model
embedding_dim = 100

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_sequence_len))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['rating'], test_size=0.2, random_state=42)

# Train the LSTM model
model.fit(X_train, y_train, epochs=5, batch_size=256, validation_data=(X_test, y_test))

# Generate embeddings from the trained LSTM model
embedding_layer = Sequential()
embedding_layer.add(model.layers[0])
embedding_layer.add(model.layers[1])
embedding_layer.compile(loss='binary_crossentropy', optimizer='Adam')

# Generate embeddings for all movies
movie_embeddings = embedding_layer.predict(padded_sequences)

# Flatten the LSTM output by averaging over timesteps
movie_embeddings_2d = np.mean(movie_embeddings, axis=1)

# Calculate cosine similarity based on 2D embeddings
cosine_sim_lstm = cosine_similarity(movie_embeddings_2d, movie_embeddings_2d)

# Function to clean movie titles (remove special characters and lowercase)
def extract_title(title):
    # Check if the title is NaN or not a string, return an empty string in that case
    if isinstance(title, str):
        title_cleaned = title.lower()
        title_cleaned = re.sub(r'\W+', ' ', title_cleaned)
        return title_cleaned.strip()
    else:
        return np.nan

# Function to recommend movies based on LSTM embeddings
def recommend_movies_lstm(movie_title, cosine_sim=cosine_sim_lstm, df=df):
    movie_title_clean = extract_title(movie_title)
    
    # Check if the movie title exists in the dataset
    if movie_title_clean in df['title'].apply(extract_title).values:
        # Get the index of the movie that matches the title
        idx = df.index[df['title'].apply(extract_title) == movie_title_clean][0]
        
        # Get the pairwise similarity scores of all movies with that movie
        sim_scores = list(enumerate(cosine_sim[idx]))
        
        # Sort the movies based on similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Get the top 3 most similar movies (excluding the input movie itself)
        recommended_movies = []
        for score in sim_scores[1:]:
            movie_data = df.iloc[score[0]]
            recommended_movies.append({
                'title': movie_data['title'],
                'film_rate': movie_data['film_rate'],
                'imdbId': movie_data['imdbId'],
                'cover_url': movie_data['cover_url'],
                'director': movie_data['director'],
                'genre': movie_data['genre'],
                'cast': movie_data['cast'],
                'overview': movie_data['overview'],
                'reviews': movie_data['reviews'],
                'tag': movie_data['tag'],
                'rating': movie_data['rating']
            })
            if len(recommended_movies) == 3:  # Limit to top 3 movies
                break
        
        # Return the top 3 most similar movies with all requested details
        return recommended_movies
    else:
        return []  # Return an empty list if the movie is not found
# Example usage with LSTM model
test_movie = 'heat'
recommended_movies_lstm = recommend_movies_lstm(test_movie)

if not recommended_movies_lstm:
    print(f"'{test_movie}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie}':")
    for movie in recommended_movies_lstm:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)


# another try 

In [ ]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics.pairwise import cosine_similarity

# Load the movie dataset
data = pd.read_csv(r'/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Preprocessing
df = pd.DataFrame(data)
df['overview'] = df['overview'].fillna('')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('(no data about the director is found)')
df['genre'] = df['genre'].fillna('')
df['tag'] = df['tag'].fillna('')
df['reviews'] = df['reviews'].fillna('')

# Feature Weighting
df['combined_features'] = (df['overview'] * 1 + ' ' +
                           df['cast'] * 3 + ' ' + 
                           df['director'] * 4 + ' ' + 
                           df['genre'] * 5 + ' ' + 
                           df['tag'] * 2 + ' ' + 
                           df['reviews'] * 1)
# Tokenization and padding
max_words = 10000
max_sequence_len = 300

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['combined_features'])
sequences = tokenizer.texts_to_sequences(df['combined_features'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='post', truncating='post')

# Define LSTM model
embedding_dim = 100

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_sequence_len))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['rating'], test_size=0.2, random_state=42)

# Train the LSTM model
model.fit(X_train, y_train, epochs=5, batch_size=256, validation_data=(X_test, y_test))

# Generate embeddings from the trained LSTM model
embedding_layer = Sequential()
embedding_layer.add(model.layers[0])
embedding_layer.add(model.layers[1])
embedding_layer.compile(loss='binary_crossentropy', optimizer='Adam')

# Generate embeddings for all movies
movie_embeddings = embedding_layer.predict(padded_sequences)

# Flatten the LSTM output by averaging over timesteps
movie_embeddings_2d = np.mean(movie_embeddings, axis=1)

# Calculate cosine similarity based on 2D embeddings
cosine_sim_lstm = cosine_similarity(movie_embeddings_2d, movie_embeddings_2d)

# Function to clean movie titles (remove special characters and lowercase)
def extract_title(title):
    if isinstance(title, str):
        title_cleaned = title.lower()
        title_cleaned = re.sub(r'\W+', ' ', title_cleaned)
        return title_cleaned.strip()
    else:
        return np.nan

# Function to recommend movies based on LSTM embeddings
def recommend_movies_lstm(movie_title, cosine_sim=cosine_sim_lstm, df=df):
    movie_title_clean = extract_title(movie_title)
    
    # Check if the movie title exists in the dataset
    if movie_title_clean in df['title'].apply(extract_title).values:
        # Get the index of the movie that matches the title
        idx = df.index[df['title'].apply(extract_title) == movie_title_clean][0]
        
        # Get the pairwise similarity scores of all movies with that movie
        sim_scores = list(enumerate(cosine_sim[idx]))
        
        # Sort the movies based on similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Get the top 3 most similar movies (excluding the input movie itself)
        recommended_movies = []
        for score in sim_scores[1:]:
            movie_data = df.iloc[score[0]]
            recommended_movies.append({
                'title': movie_data['title'],
                'film_rate': movie_data['film_rate'],
                'imdbId': movie_data['imdbId'],
                'cover_url': movie_data['cover_url'],
                'director': movie_data['director'],
                'genre': movie_data['genre'],
                'cast': movie_data['cast'],
                'overview': movie_data['overview'],
                'reviews': movie_data['reviews'],
                'tag': movie_data['tag'],
                'rating': movie_data['rating']
            })
            if len(recommended_movies) == 3:  # Limit to top 3 movies
                break
        
        # Return the top 3 most similar movies with all requested details
        return recommended_movies
    else:
        return []  # Return an empty list if the movie is not found

# Example usage with LSTM model
test_movie = 'heat'
recommended_movies_lstm = recommend_movies_lstm(test_movie)

if not recommended_movies_lstm:
    print(f"'{test_movie}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie}':")
    for movie in recommended_movies_lstm:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)


# lstm and attention  no weights

In [24]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input, Attention, Flatten
from sklearn.metrics.pairwise import cosine_similarity

# Load the movie dataset
data = pd.read_csv(r'/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Preprocessing
df = pd.DataFrame(data)
df['overview'] = df['overview'].fillna('')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('(no data about the director is found)')
df['genre'] = df['genre'].fillna('')
df['tag'] = df['tag'].fillna('')
df['reviews'] = df['reviews'].fillna('')

# Feature Weighting
df['combined_features'] = (df['overview'] * 6 + ' ' +
                           df['cast'] * 1 + ' ' + 
                           df['director'] * 4 + ' ' + 
                           df['genre'] * 5 + ' ' + 
                           df['tag'] * 1 + ' ' + 
                           df['reviews'] * 1)


# Tokenization and padding
max_words = 10000
max_sequence_len = 300

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['combined_features'])
sequences = tokenizer.texts_to_sequences(df['combined_features'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='post', truncating='post')

# Define LSTM model with Attention mechanism
embedding_dim = 100

input_layer = Input(shape=(max_sequence_len,))
embedding_layer = Embedding(max_words, embedding_dim, input_length=max_sequence_len)(input_layer)
lstm_layer_1 = LSTM(128, return_sequences=True)(embedding_layer)
dropout_layer_1 = Dropout(0.2)(lstm_layer_1)
lstm_layer_2 = LSTM(64, return_sequences=True)(dropout_layer_1)

# Attention mechanism
attention_layer = Attention()([lstm_layer_2, lstm_layer_2])
attention_output = Flatten()(attention_layer)  # Flatten the attention output

dense_layer_1 = Dense(32, activation='relu')(attention_output)
dense_layer_2 = Dense(16, activation='relu')(dense_layer_1)
output_layer = Dense(1, activation='sigmoid')(dense_layer_2)

model_with_attention = Model(inputs=input_layer, outputs=output_layer)

model_with_attention.compile(loss='catigorical_cross_entropy', optimizer='adam', metrics=['accuracy'])
model_with_attention.summary()

# Train the LSTM model with Attention mechanism
model_with_attention.fit(padded_sequences, np.ones(len(padded_sequences)), epochs=5, batch_size=256)

# Generate embeddings from the trained LSTM model with Attention mechanism
embedding_model_with_attention = Model(inputs=model_with_attention.input,
                                       outputs=model_with_attention.layers[3].output)
movie_embeddings_with_attention = embedding_model_with_attention.predict(padded_sequences)

# Flatten the LSTM output by averaging over timesteps
movie_embeddings_2d_with_attention = np.mean(movie_embeddings_with_attention, axis=1)

# Calculate cosine similarity based on 2D embeddings with Attention mechanism
cosine_sim_lstm_with_attention = cosine_similarity(movie_embeddings_2d_with_attention, movie_embeddings_2d_with_attention)

# Function to clean movie titles (remove special characters and lowercase)
def extract_title(title):
    if isinstance(title, str):
        title_cleaned = title.lower()
        title_cleaned = re.sub(r'\W+', ' ', title_cleaned)
        return title_cleaned.strip()
    else:
        return np.nan

# Function to recommend movies based on LSTM embeddings with Attention mechanism
def recommend_movies_lstm_with_attention(movie_title, cosine_sim=cosine_sim_lstm_with_attention, df=df):
    movie_title_clean = extract_title(movie_title)
    
    # Check if the movie title exists in the dataset
    if movie_title_clean in df['title'].apply(extract_title).values:
        # Get the index of the movie that matches the title
        idx = df.index[df['title'].apply(extract_title) == movie_title_clean][0]
        
        # Get the pairwise similarity scores of all movies with that movie
        sim_scores = list(enumerate(cosine_sim[idx]))
        
        # Sort the movies based on similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Get the top 3 most similar movies (excluding the input movie itself)
        recommended_movies = []
        for score in sim_scores[1:]:
            movie_data = df.iloc[score[0]]
            recommended_movies.append({
                'title': movie_data['title'],
                'film_rate': movie_data['film_rate'],
                'imdbId': movie_data['imdbId'],
                'cover_url': movie_data['cover_url'],
                'director': movie_data['director'],
                'genre': movie_data['genre'],
                'cast': movie_data['cast'],
                'overview': movie_data['overview'],
                'reviews': movie_data['reviews'],
                'tag': movie_data['tag'],
                'rating': movie_data['rating']
            })
            if len(recommended_movies) == 3:  # Limit to top 3 movies
                break
        
        # Return the top 3 most similar movies with all requested details
        return recommended_movies
    else:
        return []  # Return an empty list if the movie is not found

# Example usage with LSTM model with Attention mechanism
test_movie = 'heat'
recommended_movies_lstm_with_attention = recommend_movies_lstm_with_attention(test_movie)

if not recommended_movies_lstm_with_attention:
    print(f"'{test_movie}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie}':")
    for movie in recommended_movies_lstm_with_attention:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_35"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_10      │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_10        │ (None, 300, 100)  │  1,000,000 │ input_layer_10[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_20 (LSTM)      │ (None, 300, 128)  │    117,248 │ embedding_10[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_10          │ (None, 300, 128)  │          0 │ lstm_20[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_21 (LSTM)      │ (None, 300, 64)   │     49,408 │ dropout_10[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_10        │ (None, 300, 64)   │          0 │ lstm_21[0][0],    │
│ (Attention)         │                   │            │ lstm_21[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_7 (Flatten) │ (None, 19200)     │          0 │ attention_10[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_29 (Dense)    │ (None, 32)        │    614,432 │ flatten_7[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_30 (Dense)    │ (None, 16)        │        528 │ dense_29[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_31 (Dense)    │ (None, 1)         │         17 │ dense_30[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,781,633 (6.80 MB)

 Trainable params: 1,781,633 (6.80 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
42/42 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - accuracy: 0.9299 - loss: 0.1423
Epoch 2/5
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 3/5
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 4/5
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 5/5
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - accuracy: 1.0000 - loss: 0.0000e+00
330/330 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
Movies recommended based on 'heat':
Title: Hitchcock, Rating: 6.8, User Rating: 3.4, Tags: 
Director: Sacha Gervasi
Genre: Biography, Drama, Romance
--------------------------------------------------------------------------------
Title: Zapped!, Rating: 4.9, User Rating: 2.45, Tags: 
Director: Robert J. Rosenthal
Genre: Comedy, Fantasy, Sci-Fi
--------------------------------------------------------------------------------
Title: Kick-Ass 2, Rating: 6.5, User Rating: 3.25, Tags: 
Director: Jeff Wadlow
Genre: Action, C

In [25]:
test_movie = 'interstellar'
recommended_movies_lstm_with_attention = recommend_movies_lstm_with_attention(test_movie)

if not recommended_movies_lstm_with_attention:
    print(f"'{test_movie}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie}':")
    for movie in recommended_movies_lstm_with_attention:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)

Movies recommended based on 'interstellar':
Title: Spanglish, Rating: 6.4, User Rating: 3.2, Tags: 
Director: James L. Brooks
Genre: Comedy, Drama, Romance
--------------------------------------------------------------------------------
Title: The Whistleblower, Rating: 7.1, User Rating: 3.55, Tags: 
Director: Larysa Kondracki
Genre: Biography, Crime, Drama
--------------------------------------------------------------------------------
Title: Half Baked, Rating: 6.6, User Rating: 3.3, Tags: 
Director: Tamra Davis
Genre: Comedy, Crime
--------------------------------------------------------------------------------


# eclidian distance

In [27]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input, Attention, Flatten
from scipy.spatial.distance import euclidean

# Load the movie dataset
data = pd.read_csv(r'/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Preprocessing
df = pd.DataFrame(data)
df['overview'] = df['overview'].fillna('')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('(no data about the director is found)')
df['genre'] = df['genre'].fillna('')
df['tag'] = df['tag'].fillna('')
df['reviews'] = df['reviews'].fillna('')

# Feature Weighting
df['combined_features'] = (df['overview'] * 7 + ' ' +
                           df['cast'] * 1 + ' ' + 
                           df['director'] * 4 + ' ' + 
                           df['genre'] * 4 + ' ' + 
                           df['tag'] * 1 + ' ' + 
                           df['reviews'] * 1)


# Tokenization and padding
max_words = 10000
max_sequence_len = 300

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['combined_features'])
sequences = tokenizer.texts_to_sequences(df['combined_features'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='post', truncating='post')

# Define LSTM model with Attention mechanism
embedding_dim = 100

input_layer = Input(shape=(max_sequence_len,))
embedding_layer = Embedding(max_words, embedding_dim, input_length=max_sequence_len)(input_layer)
lstm_layer_1 = LSTM(128, return_sequences=True)(embedding_layer)
dropout_layer_1 = Dropout(0.2)(lstm_layer_1)
lstm_layer_2 = LSTM(64, return_sequences=True)(dropout_layer_1)

# Attention mechanism
attention_layer = Attention()([lstm_layer_2, lstm_layer_2])
attention_output = Flatten()(attention_layer)  # Flatten the attention output

dense_layer_1 = Dense(32, activation='relu')(attention_output)
dense_layer_2 = Dense(16, activation='relu')(dense_layer_1)
output_layer = Dense(1, activation='sigmoid')(dense_layer_2)

model_with_attention = Model(inputs=input_layer, outputs=output_layer)

model_with_attention.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_with_attention.summary()

# Train the LSTM model with Attention mechanism
model_with_attention.fit(padded_sequences, np.ones(len(padded_sequences)), epochs=5, batch_size=256)

# Generate embeddings from the trained LSTM model with Attention mechanism
embedding_model_with_attention = Model(inputs=model_with_attention.input,
                                       outputs=model_with_attention.layers[3].output)
movie_embeddings_with_attention = embedding_model_with_attention.predict(padded_sequences)

# Flatten the LSTM output by averaging over timesteps
movie_embeddings_2d_with_attention = np.mean(movie_embeddings_with_attention, axis=1)

# Calculate Euclidean distance based on 2D embeddings with Attention mechanism
def calculate_euclidean_similarity(embeddings):
    num_movies = embeddings.shape[0]
    euclidean_sim = np.zeros((num_movies, num_movies))
    for i in range(num_movies):
        for j in range(num_movies):
            euclidean_sim[i, j] = euclidean(embeddings[i], embeddings[j])
    return euclidean_sim

euclidean_sim_lstm_with_attention = calculate_euclidean_similarity(movie_embeddings_2d_with_attention)

# Function to clean movie titles (remove special characters and lowercase)
def extract_title(title):
    if isinstance(title, str):
        title_cleaned = title.lower()
        title_cleaned = re.sub(r'\W+', ' ', title_cleaned)
        return title_cleaned.strip()
    else:
        return np.nan

# Function to recommend movies based on LSTM embeddings with Attention mechanism using Euclidean distance
def recommend_movies_lstm_with_euclidean(movie_title, euclidean_sim=euclidean_sim_lstm_with_attention, df=df):
    movie_title_clean = extract_title(movie_title)
    
    # Check if the movie title exists in the dataset
    if movie_title_clean in df['title'].apply(extract_title).values:
        # Get the index of the movie that matches the title
        idx = df.index[df['title'].apply(extract_title) == movie_title_clean][0]
        
        # Get the pairwise similarity scores of all movies with that movie
        sim_scores = list(enumerate(euclidean_sim[idx]))
        
        # Sort the movies based on similarity scores (ascending order for Euclidean distance)
        sim_scores = sorted(sim_scores, key=lambda x: x[1])
        
        # Get the top 3 most similar movies (excluding the input movie itself)
        recommended_movies = []
        for score in sim_scores[1:]:
            movie_data = df.iloc[score[0]]
            recommended_movies.append({
                'title': movie_data['title'],
                'film_rate': movie_data['film_rate'],
                'imdbId': movie_data['imdbId'],
                'cover_url': movie_data['cover_url'],
                'director': movie_data['director'],
                'genre': movie_data['genre'],
                'cast': movie_data['cast'],
                'overview': movie_data['overview'],
                'reviews': movie_data['reviews'],
                'tag': movie_data['tag'],
                'rating': movie_data['rating']
            })
            if len(recommended_movies) == 3:  # Limit to top 3 movies
                break
        
        # Return the top 3 most similar movies with all requested details
        return recommended_movies
    else:
        return []  # Return an empty list if the movie is not found

# Example usage with LSTM model with Attention mechanism using Euclidean distance
test_movie_heat = 'heat'
recommended_movies_lstm_with_euclidean_heat = recommend_movies_lstm_with_euclidean(test_movie_heat)

if not recommended_movies_lstm_with_euclidean_heat:
    print(f"'{test_movie_heat}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie_heat}':")
    for movie in recommended_movies_lstm_with_euclidean_heat:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)

test_movie_interstellar = 'interstellar'
recommended_movies_lstm_with_euclidean_interstellar = recommend_movies_lstm_with_euclidean(test_movie_interstellar)

if not recommended_movies_lstm_with_euclidean_interstellar:
    print(f"'{test_movie_interstellar}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie_interstellar}':")
    for movie in recommended_movies_lstm_with_euclidean_interstellar:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_43"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_12      │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_12        │ (None, 300, 100)  │  1,000,000 │ input_layer_12[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_24 (LSTM)      │ (None, 300, 128)  │    117,248 │ embedding_12[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_12          │ (None, 300, 128)  │          0 │ lstm_24[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_25 (LSTM)      │ (None, 300, 64)   │     49,408 │ dropout_12[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_12        │ (None, 300, 64)   │          0 │ lstm_25[0][0],    │
│ (Attention)         │                   │            │ lstm_25[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_9 (Flatten) │ (None, 19200)     │          0 │ attention_12[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_35 (Dense)    │ (None, 32)        │    614,432 │ flatten_9[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_36 (Dense)    │ (None, 16)        │        528 │ dense_35[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_37 (Dense)    │ (None, 1)         │         17 │ dense_36[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,781,633 (6.80 MB)

 Trainable params: 1,781,633 (6.80 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
42/42 ━━━━━━━━━━━━━━━━━━━━ 6s 67ms/step - accuracy: 0.9194 - loss: 0.1263
Epoch 2/5
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 3/5
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 4/5
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 5/5
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 1.0000 - loss: 0.0000e+00
330/330 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
Movies recommended based on 'heat':
Title: Brothers, Rating: 7.1, User Rating: 3.55, Tags: 
Director: Susanne Bier
Genre: Drama, War
--------------------------------------------------------------------------------
Title: Brothers, Rating: 7.1, User Rating: 3.55, Tags: 
Director: Jim Sheridan
Genre: Drama, Thriller, War
--------------------------------------------------------------------------------
Title: Hitchcock, Rating: 6.8, User Rating: 3.4, Tags: 
Director: Sacha Gervasi
Genre: Biography, Drama, Romance
--------

# PCA

In [28]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input, Attention, Flatten
from sklearn.decomposition import PCA
from scipy.spatial.distance import euclidean

# Load the movie dataset
data = pd.read_csv(r'/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Preprocessing
df = pd.DataFrame(data)
df['overview'] = df['overview'].fillna('')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('(no data about the director is found)')
df['genre'] = df['genre'].fillna('')
df['tag'] = df['tag'].fillna('')
df['reviews'] = df['reviews'].fillna('')

# data cleaning
df['overview'] = df['overview'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['cast'] = df['cast'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['director'] = df['director'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['genre'] = df['genre'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['tag'] = df['tag'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['reviews'] = df['reviews'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))

# Feature Weighting
df['combined_features'] = (df['overview'] * 7 + ' ' +
                           df['cast'] * 1 + ' ' + 
                           df['director'] * 8 + ' ' + 
                           df['genre'] * 10 + ' ' + 
                           df['tag'] * 1 + ' ' + 
                           df['reviews'] * 1)

# Tokenization and padding
max_words = 10000
max_sequence_len = 500

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['combined_features'])
sequences = tokenizer.texts_to_sequences(df['combined_features'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='post', truncating='post')

# Define LSTM model with Attention mechanism
embedding_dim = 100

input_layer = Input(shape=(max_sequence_len,))
embedding_layer = Embedding(max_words, embedding_dim, input_length=max_sequence_len)(input_layer)
lstm_layer_1 = LSTM(128, return_sequences=True)(embedding_layer)
dropout_layer_1 = Dropout(0.2)(lstm_layer_1)
lstm_layer_2 = LSTM(64, return_sequences=True)(dropout_layer_1)

# Attention mechanism
attention_layer = Attention()([lstm_layer_2, lstm_layer_2])
attention_output = Flatten()(attention_layer)  # Flatten the attention output

dense_layer_1 = Dense(32, activation='relu')(attention_output)
dense_layer_2 = Dense(16, activation='relu')(dense_layer_1)
output_layer = Dense(1, activation='sigmoid')(dense_layer_2)

model_with_attention = Model(inputs=input_layer, outputs=output_layer)

model_with_attention.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_with_attention.summary()

# Train the LSTM model with Attention mechanism
model_with_attention.fit(padded_sequences, np.ones(len(padded_sequences)), epochs=5, batch_size=256)

# Generate embeddings from the trained LSTM model with Attention mechanism
embedding_model_with_attention = Model(inputs=model_with_attention.input,
                                       outputs=model_with_attention.layers[3].output)
movie_embeddings_with_attention = embedding_model_with_attention.predict(padded_sequences)

# Flatten the LSTM output by averaging over timesteps
movie_embeddings_2d_with_attention = np.mean(movie_embeddings_with_attention, axis=1)

# Reduce dimensionality of embeddings using PCA
pca = PCA(n_components=50)  # Adjust the number of components as needed
reduced_embeddings = pca.fit_transform(movie_embeddings_2d_with_attention)

# Calculate Euclidean distance based on reduced embeddings
def calculate_euclidean_similarity(embeddings):
    num_movies = embeddings.shape[0]
    euclidean_sim = np.zeros((num_movies, num_movies))
    for i in range(num_movies):
        for j in range(num_movies):
            euclidean_sim[i, j] = euclidean(embeddings[i], embeddings[j])
    return euclidean_sim

euclidean_sim_lstm_with_attention_reduced = calculate_euclidean_similarity(reduced_embeddings)

# Function to clean movie titles (remove special characters and lowercase)
def extract_title(title):
    if isinstance(title, str):
        title_cleaned = title.lower()
        title_cleaned = re.sub(r'\W+', ' ', title_cleaned)
        return title_cleaned.strip()
    else:
        return np.nan

# Function to recommend movies based on reduced embeddings with Euclidean distance
def recommend_movies_lstm_with_euclidean_reduced(movie_title, euclidean_sim=euclidean_sim_lstm_with_attention_reduced, df=df):
    movie_title_clean = extract_title(movie_title)
    
    # Check if the movie title exists in the dataset
    if movie_title_clean in df['title'].apply(extract_title).values:
        # Get the index of the movie that matches the title
        idx = df.index[df['title'].apply(extract_title) == movie_title_clean][0]
        
        # Get the pairwise similarity scores of all movies with that movie
        sim_scores = list(enumerate(euclidean_sim[idx]))
        
        # Sort the movies based on similarity scores (ascending order for Euclidean distance)
        sim_scores = sorted(sim_scores, key=lambda x: x[1])
        
        # Get the top 3 most similar movies (excluding the input movie itself)
        recommended_movies = []
        for score in sim_scores[1:]:
            movie_data = df.iloc[score[0]]
            recommended_movies.append({
                'title': movie_data['title'],
                'film_rate': movie_data['film_rate'],
                'imdbId': movie_data['imdbId'],
                'cover_url': movie_data['cover_url'],
                'director': movie_data['director'],
                'genre': movie_data['genre'],
                'cast': movie_data['cast'],
                'overview': movie_data['overview'],
                'reviews': movie_data['reviews'],
                'tag': movie_data['tag'],
                'rating': movie_data['rating']
            })
            if len(recommended_movies) == 3:  # Limit to top 3 movies
                break
        
        # Return the top 3 most similar movies with all requested details
        return recommended_movies
    else:
        return []  # Return an empty list if the movie is not found

# Example usage with reduced embeddings and Euclidean distance
test_movie_heat = 'heat'
recommended_movies_lstm_with_euclidean_reduced_heat = recommend_movies_lstm_with_euclidean_reduced(test_movie_heat)

if not recommended_movies_lstm_with_euclidean_reduced_heat:
    print(f"'{test_movie_heat}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie_heat}':")
    for movie in recommended_movies_lstm_with_euclidean_reduced_heat:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)

test_movie_interstellar = 'interstellar'
recommended_movies_lstm_with_euclidean_reduced_interstellar = recommend_movies_lstm_with_euclidean_reduced(test_movie_interstellar)

if not recommended_movies_lstm_with_euclidean_reduced_interstellar:
    print(f"'{test_movie_interstellar}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie_interstellar}':")
    for movie in recommended_movies_lstm_with_euclidean_reduced_interstellar:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_47"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_13      │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_13        │ (None, 300, 100)  │  1,000,000 │ input_layer_13[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_26 (LSTM)      │ (None, 300, 128)  │    117,248 │ embedding_13[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_13          │ (None, 300, 128)  │          0 │ lstm_26[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_27 (LSTM)      │ (None, 300, 64)   │     49,408 │ dropout_13[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_13        │ (None, 300, 64)   │          0 │ lstm_27[0][0],    │
│ (Attention)         │                   │            │ lstm_27[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_10          │ (None, 19200)     │          0 │ attention_13[0][… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_38 (Dense)    │ (None, 32)        │    614,432 │ flatten_10[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_39 (Dense)    │ (None, 16)        │        528 │ dense_38[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_40 (Dense)    │ (None, 1)         │         17 │ dense_39[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,781,633 (6.80 MB)

 Trainable params: 1,781,633 (6.80 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
42/42 ━━━━━━━━━━━━━━━━━━━━ 5s 67ms/step - accuracy: 0.8270 - loss: 0.1749
Epoch 2/5
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 3/5
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 4/5
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 5/5
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 1.0000 - loss: 0.0000e+00
330/330 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
Movies recommended based on 'heat':
Title: Jane Eyre, Rating: 7.5, User Rating: 3.75, Tags: 
Director: Franco Zeffirelli
Genre: Drama, Romance
--------------------------------------------------------------------------------
Title: Jane Eyre, Rating: 7.5, User Rating: 3.75, Tags: 
Director: Robert Stevenson
Genre: Drama, Romance
--------------------------------------------------------------------------------
Title: Jane Eyre, Rating: 7.5, User Rating: 3.75, Tags: 
Director: Delbert Mann
Genre: Drama
-------------------

In [29]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input, Attention, Flatten
from sklearn.decomposition import PCA
from scipy.spatial.distance import euclidean

# Load the movie dataset
data = pd.read_csv(r'/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Preprocessing
df = pd.DataFrame(data)
df['overview'] = df['overview'].fillna('')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('(no data about the director is found)')
df['genre'] = df['genre'].fillna('')
df['tag'] = df['tag'].fillna('')
df['reviews'] = df['reviews'].fillna('')

# Clean text data
df['overview'] = df['overview'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['cast'] = df['cast'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['director'] = df['director'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['genre'] = df['genre'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['tag'] = df['tag'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['reviews'] = df['reviews'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))

# Feature Weighting
df['combined_features'] = (df['overview'] * 7 + ' ' +
                           df['cast'] * 2 + ' ' + 
                           df['director'] * 5 + ' ' + 
                           df['genre'] * 6 + ' ' + 
                           df['tag'] * 1 + ' ' + 
                           df['reviews'] * 1)

# Tokenization and padding
max_words = 10000
max_sequence_len = 300

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['combined_features'])
sequences = tokenizer.texts_to_sequences(df['combined_features'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='post', truncating='post')

# Define LSTM model with Attention mechanism
embedding_dim = 100

input_layer = Input(shape=(max_sequence_len,))
embedding_layer = Embedding(max_words, embedding_dim, input_length=max_sequence_len)(input_layer)
lstm_layer_1 = LSTM(256, return_sequences=True)(embedding_layer)
dropout_layer_1 = Dropout(0.3)(lstm_layer_1)
lstm_layer_2 = LSTM(128, return_sequences=True)(dropout_layer_1)
dropout_layer_2 = Dropout(0.3)(lstm_layer_2)
lstm_layer_3 = LSTM(64, return_sequences=True)(dropout_layer_2)

# Attention mechanism
attention_layer = Attention()([lstm_layer_3, lstm_layer_3])
attention_output = Flatten()(attention_layer)  # Flatten the attention output

dense_layer_1 = Dense(64, activation='relu')(attention_output)
dense_layer_2 = Dense(32, activation='relu')(dense_layer_1)
output_layer = Dense(1, activation='sigmoid')(dense_layer_2)

model_with_attention = Model(inputs=input_layer, outputs=output_layer)

model_with_attention.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_with_attention.summary()

# Train the LSTM model with Attention mechanism
model_with_attention.fit(padded_sequences, np.ones(len(padded_sequences)), epochs=5, batch_size=256)

# Generate embeddings from the trained LSTM model with Attention mechanism
embedding_model_with_attention = Model(inputs=model_with_attention.input,
                                       outputs=model_with_attention.layers[3].output)
movie_embeddings_with_attention = embedding_model_with_attention.predict(padded_sequences)

# Flatten the LSTM output by averaging over timesteps
movie_embeddings_2d_with_attention = np.mean(movie_embeddings_with_attention, axis=1)

# Reduce dimensionality of embeddings using PCA
pca = PCA(n_components=50)  # Adjust the number of components as needed
reduced_embeddings = pca.fit_transform(movie_embeddings_2d_with_attention)

# Calculate Euclidean distance based on reduced embeddings
def calculate_euclidean_similarity(embeddings):
    num_movies = embeddings.shape[0]
    euclidean_sim = np.zeros((num_movies, num_movies))
    for i in range(num_movies):
        for j in range(num_movies):
            euclidean_sim[i, j] = euclidean(embeddings[i], embeddings[j])
    return euclidean_sim

euclidean_sim_lstm_with_attention_reduced = calculate_euclidean_similarity(reduced_embeddings)

# Function to clean movie titles (remove special characters and lowercase)
def extract_title(title):
    if isinstance(title, str):
        title_cleaned = title.lower()
        title_cleaned = re.sub(r'\W+', ' ', title_cleaned)
        return title_cleaned.strip()
    else:
        return np.nan

# Function to recommend movies based on reduced embeddings with Euclidean distance
def recommend_movies_lstm_with_euclidean_reduced(movie_title, euclidean_sim=euclidean_sim_lstm_with_attention_reduced, df=df):
    movie_title_clean = extract_title(movie_title)
    
    # Check if the movie title exists in the dataset
    if movie_title_clean in df['title'].apply(extract_title).values:
        # Get the index of the movie that matches the title
        idx = df.index[df['title'].apply(extract_title) == movie_title_clean][0]
        
        # Get the pairwise similarity scores of all movies with that movie
        sim_scores = list(enumerate(euclidean_sim[idx]))
        
        # Sort the movies based on similarity scores (ascending order for Euclidean distance)
        sim_scores = sorted(sim_scores, key=lambda x: x[1])
        
        # Get the top 3 most similar movies (excluding the input movie itself)
        recommended_movies = []
        for score in sim_scores[1:]:
            movie_data = df.iloc[score[0]]
            recommended_movies.append({
                'title': movie_data['title'],
                'film_rate': movie_data['film_rate'],
                'imdbId': movie_data['imdbId'],
                'cover_url': movie_data['cover_url'],
                'director': movie_data['director'],
                'genre': movie_data['genre'],
                'cast': movie_data['cast'],
                'overview': movie_data['overview'],
                'reviews': movie_data['reviews'],
                'tag': movie_data['tag'],
                'rating': movie_data['rating']
            })
            if len(recommended_movies) == 3:  # Limit to top 3 movies
                break
        
        # Return the top 3 most similar movies with all requested details
        return recommended_movies
    else:
        return []  # Return an empty list if the movie is not found

# Example usage with reduced embeddings and Euclidean distance
test_movie_heat = 'heat'
recommended_movies_lstm_with_euclidean_reduced_heat = recommend_movies_lstm_with_euclidean_reduced(test_movie_heat)

if not recommended_movies_lstm_with_euclidean_reduced_heat:
    print(f"'{test_movie_heat}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie_heat}':")
    for movie in recommended_movies_lstm_with_euclidean_reduced_heat:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_51"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_14      │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_14        │ (None, 300, 100)  │  1,000,000 │ input_layer_14[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_28 (LSTM)      │ (None, 300, 256)  │    365,568 │ embedding_14[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_14          │ (None, 300, 256)  │          0 │ lstm_28[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_29 (LSTM)      │ (None, 300, 128)  │    197,120 │ dropout_14[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_15          │ (None, 300, 128)  │          0 │ lstm_29[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_30 (LSTM)      │ (None, 300, 64)   │     49,408 │ dropout_15[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_14        │ (None, 300, 64)   │          0 │ lstm_30[0][0],    │
│ (Attention)         │                   │            │ lstm_30[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_11          │ (None, 19200)     │          0 │ attention_14[0][… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_41 (Dense)    │ (None, 64)        │  1,228,864 │ flatten_11[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_42 (Dense)    │ (None, 32)        │      2,080 │ dense_41[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_43 (Dense)    │ (None, 1)         │         33 │ dense_42[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,843,073 (10.85 MB)

 Trainable params: 2,843,073 (10.85 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
42/42 ━━━━━━━━━━━━━━━━━━━━ 9s 131ms/step - accuracy: 0.9909 - loss: 0.1125
Epoch 2/5
42/42 ━━━━━━━━━━━━━━━━━━━━ 6s 131ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 3/5
42/42 ━━━━━━━━━━━━━━━━━━━━ 6s 131ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 4/5
42/42 ━━━━━━━━━━━━━━━━━━━━ 6s 131ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 5/5
42/42 ━━━━━━━━━━━━━━━━━━━━ 6s 131ms/step - accuracy: 1.0000 - loss: 0.0000e+00
330/330 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
Movies recommended based on 'heat':
Title: Son of a Gun, Rating: 6.4, User Rating: 3.2, Tags: 
Director: julius avery
Genre: action crime drama thriller
Title: Because I Said So, Rating: 5.6, User Rating: 2.8, Tags: 
Director: michael lehmann
Genre: comedy romance
Title: About Adam, Rating: 5.8, User Rating: 2.9, Tags: 
Director: gerard stembridge
Genre: comedy romance


In [30]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input, Attention, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.decomposition import PCA
from scipy.spatial.distance import euclidean

# Load the movie dataset
data = pd.read_csv(r'/kaggle/input/movies-updated/movies_with_tags_rating.csv')

# Preprocessing
df = pd.DataFrame(data)
df['overview'] = df['overview'].fillna('')
df['cast'] = df['cast'].fillna('')
df['director'] = df['director'].fillna('(no data about the director is found)')
df['genre'] = df['genre'].fillna('')
df['tag'] = df['tag'].fillna('')
df['reviews'] = df['reviews'].fillna('')

# Clean text data
df['overview'] = df['overview'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['cast'] = df['cast'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['director'] = df['director'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['genre'] = df['genre'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['tag'] = df['tag'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))
df['reviews'] = df['reviews'].apply(lambda x: re.sub(r'\W+', ' ', x.lower()))

# Feature Weighting
df['combined_features'] = (df['overview'] * 7 + ' ' +
                           df['cast'] * 2 + ' ' + 
                           df['director'] * 5 + ' ' + 
                           df['genre'] * 6 + ' ' + 
                           df['tag'] * 1 + ' ' + 
                           df['reviews'] * 1)

# Tokenization and padding
max_words = 10000
max_sequence_len = 300

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['combined_features'])
sequences = tokenizer.texts_to_sequences(df['combined_features'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='post', truncating='post')

# Define LSTM model with Attention mechanism
embedding_dim = 100

input_layer = Input(shape=(max_sequence_len,))
embedding_layer = Embedding(max_words, embedding_dim, input_length=max_sequence_len)(input_layer)
lstm_layer_1 = LSTM(256, return_sequences=True, kernel_regularizer=l2(0.01))(embedding_layer)
dropout_layer_1 = Dropout(0.5)(lstm_layer_1)
lstm_layer_2 = LSTM(128, return_sequences=True, kernel_regularizer=l2(0.01))(dropout_layer_1)
dropout_layer_2 = Dropout(0.5)(lstm_layer_2)
lstm_layer_3 = LSTM(64, return_sequences=True, kernel_regularizer=l2(0.01))(dropout_layer_2)

# Attention mechanism
attention_layer = Attention()([lstm_layer_3, lstm_layer_3])
attention_output = Flatten()(attention_layer)  # Flatten the attention output

dense_layer_1 = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(attention_output)
dense_layer_2 = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(dense_layer_1)
output_layer = Dense(1, activation='sigmoid')(dense_layer_2)

model_with_attention = Model(inputs=input_layer, outputs=output_layer)

model_with_attention.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_with_attention.summary()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the LSTM model with Attention mechanism
model_with_attention.fit(padded_sequences, np.ones(len(padded_sequences)), epochs=20, batch_size=256, validation_split=0.2, callbacks=[early_stopping])

# Generate embeddings from the trained LSTM model with Attention mechanism
embedding_model_with_attention = Model(inputs=model_with_attention.input,
                                       outputs=model_with_attention.layers[3].output)
movie_embeddings_with_attention = embedding_model_with_attention.predict(padded_sequences)

# Flatten the LSTM output by averaging over timesteps
movie_embeddings_2d_with_attention = np.mean(movie_embeddings_with_attention, axis=1)

# Reduce dimensionality of embeddings using PCA
pca = PCA(n_components=50)  # Adjust the number of components as needed
reduced_embeddings = pca.fit_transform(movie_embeddings_2d_with_attention)

# Calculate Euclidean distance based on reduced embeddings
def calculate_euclidean_similarity(embeddings):
    num_movies = embeddings.shape[0]
    euclidean_sim = np.zeros((num_movies, num_movies))
    for i in range(num_movies):
        for j in range(num_movies):
            euclidean_sim[i, j] = euclidean(embeddings[i], embeddings[j])
    return euclidean_sim

euclidean_sim_lstm_with_attention_reduced = calculate_euclidean_similarity(reduced_embeddings)

# Function to clean movie titles (remove special characters and lowercase)
def extract_title(title):
    if isinstance(title, str):
        title_cleaned = title.lower()
        title_cleaned = re.sub(r'\W+', ' ', title_cleaned)
        return title_cleaned.strip()
    else:
        return np.nan

# Function to recommend movies based on reduced embeddings with Euclidean distance
def recommend_movies_lstm_with_euclidean_reduced(movie_title, euclidean_sim=euclidean_sim_lstm_with_attention_reduced, df=df):
    movie_title_clean = extract_title(movie_title)
    
    # Check if the movie title exists in the dataset
    if movie_title_clean in df['title'].apply(extract_title).values:
        # Get the index of the movie that matches the title
        idx = df.index[df['title'].apply(extract_title) == movie_title_clean][0]
        
        # Get the pairwise similarity scores of all movies with that movie
        sim_scores = list(enumerate(euclidean_sim[idx]))
        
        # Sort the movies based on similarity scores (ascending order for Euclidean distance)
        sim_scores = sorted(sim_scores, key=lambda x: x[1])
        
        # Get the top 3 most similar movies (excluding the input movie itself)
        recommended_movies = []
        for score in sim_scores[1:]:
            movie_data = df.iloc[score[0]]
            recommended_movies.append({
                'title': movie_data['title'],
                'film_rate': movie_data['film_rate'],
                'imdbId': movie_data['imdbId'],
                'cover_url': movie_data['cover_url'],
                'director': movie_data['director'],
                'genre': movie_data['genre'],
                'cast': movie_data['cast'],
                'overview': movie_data['overview'],
                'reviews': movie_data['reviews'],
                'tag': movie_data['tag'],
                'rating': movie_data['rating']
            })
            if len(recommended_movies) == 3:  # Limit to top 3 movies
                break
        
        # Return the top 3 most similar movies with all requested details
        return recommended_movies
    else:
        return []  # Return an empty list if the movie is not found

# Example usage with reduced embeddings and Euclidean distance
test_movie_heat = 'heat'
recommended_movies_lstm_with_euclidean_reduced_heat = recommend_movies_lstm_with_euclidean_reduced(test_movie_heat)

if not recommended_movies_lstm_with_euclidean_reduced_heat:
    print(f"'{test_movie_heat}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie_heat}':")
    for movie in recommended_movies_lstm_with_euclidean_reduced_heat:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)

test_movie_interstellar = 'interstellar'
recommended_movies_lstm_with_euclidean_reduced_interstellar = recommend_movies_lstm_with_euclidean_reduced(test_movie_interstellar)

if not recommended_movies_lstm_with_euclidean_reduced_interstellar:
    print(f"'{test_movie_interstellar}' is not listed in our database.")
else:
    print(f"Movies recommended based on '{test_movie_interstellar}':")
    for movie in recommended_movies_lstm_with_euclidean_reduced_interstellar:
        print(f"Title: {movie['title']}, Rating: {movie['film_rate']}, User Rating: {movie['rating']}, Tags: {movie['tag']}")
        print(f"Director: {movie['director']}")
        print(f"Genre: {movie['genre']}")
        print("-" * 80)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_55"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_15      │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_15        │ (None, 300, 100)  │  1,000,000 │ input_layer_15[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_31 (LSTM)      │ (None, 300, 256)  │    365,568 │ embedding_15[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_16          │ (None, 300, 256)  │          0 │ lstm_31[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_32 (LSTM)      │ (None, 300, 128)  │    197,120 │ dropout_16[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_17          │ (None, 300, 128)  │          0 │ lstm_32[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_33 (LSTM)      │ (None, 300, 64)   │     49,408 │ dropout_17[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_15        │ (None, 300, 64)   │          0 │ lstm_33[0][0],    │
│ (Attention)         │                   │            │ lstm_33[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_12          │ (None, 19200)     │          0 │ attention_15[0][… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_44 (Dense)    │ (None, 64)        │  1,228,864 │ flatten_12[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_45 (Dense)    │ (None, 32)        │      2,080 │ dense_44[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_46 (Dense)    │ (None, 1)         │         33 │ dense_45[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,843,073 (10.85 MB)

 Trainable params: 2,843,073 (10.85 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 9s 163ms/step - accuracy: 0.9991 - loss: 6.5478 - val_accuracy: 1.0000 - val_loss: 2.4152
Epoch 2/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 5s 148ms/step - accuracy: 1.0000 - loss: 1.8413 - val_accuracy: 1.0000 - val_loss: 0.7248
Epoch 3/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 5s 149ms/step - accuracy: 1.0000 - loss: 0.5669 - val_accuracy: 1.0000 - val_loss: 0.2569
Epoch 4/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 5s 148ms/step - accuracy: 1.0000 - loss: 0.2104 - val_accuracy: 1.0000 - val_loss: 0.1135
Epoch 5/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 5s 148ms/step - accuracy: 1.0000 - loss: 0.0966 - val_accuracy: 1.0000 - val_loss: 0.0586
Epoch 6/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 5s 148ms/step - accuracy: 1.0000 - loss: 0.0510 - val_accuracy: 1.0000 - val_loss: 0.0330
Epoch 7/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 5s 148ms/step - accuracy: 1.0000 - loss: 0.0290 - val_accuracy: 1.0000 - val_loss: 0.0193
Epoch 8/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 5s 148ms/step - accuracy: 1.0000 - loss: 0.0171 - val_accuracy: 1.